In [ ]:
# !git clone https://github.com/amitkaps/recommendation.git
# !mkdir grocery && cp /content/recommendation/Groceries/data/*.csv grocery
# %cd grocery

In [53]:
# !cp /content/grocery/*.csv /content/drive/MyDrive/TempData

In [8]:
import pandas as pd
from scipy.spatial.distance import cosine

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
data = pd.read_csv("groceries.csv")
data[:10]

,Person,item
0,1,citrus fruit
1,1,semi-finished bread
2,1,margarine
3,1,ready soups
4,2,tropical fruit
5,2,yogurt
6,2,coffee
7,3,whole milk
8,4,pip fruit
9,4,yogurt


In [7]:
print("number of unique items: %d"%data.item.nunique())

number of unique items: 169


In [10]:
data["Quantity"] = 1
dataWide = data.pivot("Person", "item", "Quantity")
dataWide.fillna(0, inplace=True)
dataWide.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen chicken,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,kitchen utensil,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,make up remover,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic products,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,preservation products,processed cheese,prosecco,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,sound storage medium,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

## Item-based CF

In [13]:
data_ib = dataWide.copy()
data_ib = data_ib.reset_index(drop=True)
data_ibs = pd.DataFrame(index=data_ib.columns, columns=data_ib.columns)
data_ibs.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen chicken,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,kitchen utensil,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,make up remover,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic products,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,preservation products,processed cheese,prosecco,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,sound storage medium,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
item,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Instant food products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UHT-milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abrasive cleaner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [15]:
for i in range(0,len(data_ibs.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)) :
      # Fill in placeholder with cosine similarities
      data_ibs.iloc[i,j] = 1-cosine(data_ib.iloc[:,i],data_ib.iloc[:,j])
data_ibs.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen chicken,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,kitchen utensil,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,make up remover,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic products,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,preservation products,processed cheese,prosecco,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,sound storage medium,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
item,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Instant food products,1,0.0248112,0,0,0,0,0,0.0255878,0,0.0396234,0.0186653,0.0281272,0.023987,0.0341249,0,0.0311799,0.0578322,0,0.00986767,0.023987,0.0131233,0.00814085,0.0832335,0.0397779,0.0109278,0,0,0.0156398,0.0219074,0.0203721,0.0238518,0.0433777,0,0.0106311,0,0.0235417,0.011195,0,0,0,0.0170913,0.0638946,0.0159111,0,0.0149022,0.0235559,0.0327352,0.0221716,0.0171078,0,0.0450396,0.0145248,0.0140636,0,0.034415,0.01114,0,0.0467167,0,0.0218556,0.0104915,0,0.0269429,0.0123494,0.0362121,0.0379747,0.0151707,0,0.0140636,0.186653,0.0289893,0.0266838,0.0290496,0.0496981,0.0437534,0.0358665,0,0.0154543,0.034721,0.0146474,0.0562544,0,0,0,0,0,0.0117299,0,0,0.0328151,0.0355784,0.0352971,0,0.0134715,0.0207145,0.0396618,0.0441717,0.0202072,0.0195853,0.0270889,0.0257689,0,0,0.0696355,0,0.055489,0.0532489,0,0.0117941,0.0169613,0.0412477,0.0267047,0.0472493,0,0.0212622,0,0.0440618,0,0.0234597,0,0.0245514,0.0259828,0.011195,0.0608408,0.0652894,0,0,0,0.0218556,0.0466665,0,0.0296101,0,0.0170585,0.0397573,0.0380349,0.0362366,0.0205412,0,0.0516188,0,0.0153105,0,0.0412354,0,0.0205793,0.0122757,0.0195197,0,0,0.0157544,0.0321819,0.0308272,0.0238518,0.0198889,0.0365027,0,0,0.0525337,0.0251577,0.0140636,0.0173605,0.0296613,0,0.044236,0,0.0673304,0.0425243,0.0409311
UHT-milk,0.0248112,1,0,0.0389841,0,0,0,0.0376158,0.0212202,0.0339786,0.0518295,0.0275659,0.0470165,0.120398,0.0344406,0.0873075,0.0543165,0.0598423,0.0483538,0.0235083,0.0385842,0.0378974,0.0407862,0.019492,0.0481939,0.0546482,0.0147346,0.0229915,0.059043,0.0399312,0.00584396,0.0869566,0.00779681,0.0416757,0.0235083,0.0969021,0.0493724,0,0.0106101,0.0152908,0.0781678,0.0553942,0.00779681,0.0284699,0.0292095,0.0865719,0.0240615,0.0380261,0.0251496,0.0180461,0.0882815,0.0284699,0.00689147,0.0204754,0.0252962,0.0218354,0,0

In [17]:
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:10] = data_ibs.iloc[0:,i].sort_values(ascending=False)[:10].index
    
data_neighbours

,1,2,3,4,5,6,7,8,9,10
item,,,,,,,,,,
Instant food products,Instant food products,hamburger meat,canned fish,other vegetables,whole milk,root vegetables,curd,rolls/buns,butter,kitchen utensil
UHT-milk,UHT-milk,bottled water,yogurt,other vegetables,soda,coffee,margarine,domestic eggs,brown bread,citrus fruit
abrasive cleaner,abrasive cleaner,preservation products,cleaner,curd cheese,root vegetables,dish cleaner,other vegetables,salad dressing,rice,berries
artif. sweetener,artif. sweetener,potato products,salad dressing,skin care,candles,flour,rum,yogurt,frankfurter,mustard
baby cosmetics,baby cosmetics,cream,dish cleaner,cookware,syrup,kitchen towels,soups,butter milk,oil,sweet spreads
baby food,baby food,finished products,soups,cake bar,pasta,soft cheese,butter milk,dessert,salty snack,waffles
bags,bags,tidbits,frozen potato products,pickled vegetables,frozen vegetables,napkins,pork,fruit/vegetable juice,pip fruit,pastry
baking powder,baking powder,whole milk,sugar,whipped/sour cream,other vegetables,cooking chocolate,flour,margarine,domestic eggs,yogurt
bathroom cleaner,bathroom cleaner,cleaner,liver loaf,decalcifier,root vegetables,soda,other vegetables,liquor (appetizer),berries,napkins


## User-based CF


The process for creating a User Based recommendation system is as follows:
1. Have Item-Based similarity matrix
2. Check which items the user has consumed
3. For each item the user has consumed, get the top X neighbours
4. Get the consumption record of the user for each neighbour.
5. Compute similarity score
6. Recommend the items with the highest score

In [23]:
data_neighbours.loc['Instant food products'][1:10]

# for i in range(0, len(data_sims11.index)):
#     for j in range(1, len(data_sims11.columns)):
#         user = data_sims11.index[i]
#         product = data_sims11.columns[j]
#         print(user, product)

2       hamburger meat
3          canned fish
4     other vegetables
5           whole milk
6      root vegetables
7                 curd
8           rolls/buns
9               butter
10     kitchen utensil
Name: Instant food products, dtype: object

In [25]:
# Helper function to get similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

data_sims1 = dataWide.reset_index()

# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=data_sims1.index,columns=data_sims1.columns)
data_sims.iloc[:,:1] = data_sims1.iloc[:,:1]

data_sims12 = data_sims1.iloc[:50,:]
data_sims11 = data_sims.iloc[:50,:]

# Need to run this for only 50 users. Might be slow beyond that. 
for i in range(0, len(data_sims11.index)):
    for j in range(1, len(data_sims11.columns)):
        user = data_sims11.index[i]
        product = data_sims11.columns[j]
 
        if data_sims12.iloc[i][j] == 1:
            data_sims11.iloc[i][j] = 0
        else:
            product_top_names = data_neighbours.loc[product][1:10]
            product_top_sims = data_ibs.loc[product].sort_values(ascending=False)[1:10]
            user_purchases = data_ib.loc[user,product_top_names] 
            data_sims11.iloc[i][j] = getScore(user_purchases,product_top_sims)

# Get the top products
data_recommend = pd.DataFrame(index=data_sims.index, columns=['Person','1','2','3','4','5','6'])
data_recommend.iloc[0:,0] = data_sims.iloc[:,0]

# Instead of top product scores, we want to see names
for i in range(0,len(data_sims.index)):
    data_recommend.iloc[i,1:] = data_sims.iloc[i,:].sort_values(ascending=False).iloc[1:7,].index.transpose()

# Print a sample
print(data_recommend.iloc[:10,:4])

  Person                  1               2                  3
0      1            candles      hair spray                oil
1      2  seasonal products       pip fruit          beverages
2      3            cereals            curd      domestic eggs
3      4        butter milk  tropical fruit        canned fish
4      5             coffee         cereals          chocolate
5      6               curd         cleaner      domestic eggs
6      7        frankfurter         sausage      spread cheese
7      8     red/blush wine          dishes      bottled water
8      9           prosecco        dog food  finished products
9     10               soap            curd      domestic eggs


## Market Basket Analyis

In [27]:
import itertools
import sys
import time
import pandas as pd

In [38]:
def tokenize(file_name):
    # Assumes that sentences are separated by a single '\n'.
    # Assumes that words are separated by a single ' '.
    # Tokenizes each sentence, removes duplicate tokens, sorts tokens.
    return [sorted(list(set(e.split(",")))) for e in
            open(file_name).read().strip(",").split('\n')]

In [39]:
def frequent_itemsets(sentences):
    # Counts sets with Apriori algorithm.
    SUPP_THRESHOLD = 100
    supps = []
    
    supp = {}
    for sentence in sentences:
        for key in sentence:
            if key in supp:
                supp[key] += 1
            else:
                supp[key] = 1
    print("|C1| = " + str(len(supp)))
    supps.append({k:v for k,v in supp.items() if v >= SUPP_THRESHOLD})
    print("|L1| = " + str(len(supps[0])))

    supp = {}
    for sentence in sentences:
        for combination in itertools.combinations(sentence, 2):
            if combination[0] in supps[0] and combination[1] in supps[0]:
                key = ','.join(combination)
                if key in supp:
                    supp[key] += 1
                else:
                    supp[key] = 1
    print("|C2| = " + str(len(supp)))
    supps.append({k:v for k,v in supp.items() if v >= SUPP_THRESHOLD})
    print("|L2| = " + str(len(supps[1])))
 
    supp = {}
    for sentence in sentences:
        for combination in itertools.combinations(sentence, 3):
            if (combination[0]+','+combination[1] in supps[1] and
                    combination[0]+','+combination[2] in supps[1] and
                    combination[1]+','+combination[2] in supps[1]):
                key = ','.join(combination)
                if key in supp:
                    supp[key] += 1
                else:
                    supp[key] = 1
    print("|C3| = " + str(len(supp)))
    supps.append({k:v for k,v in supp.items() if v >= SUPP_THRESHOLD})
    print("|L3| = " + str(len(supps[2])))
 
    return supps

In [40]:
def measures(supp_ab, supp_a, supp_b, transaction_count):
    # Assumes A -> B, where A and B are sets.
    conf = float(supp_ab) / float(supp_a)
    s = float(supp_b) / float(transaction_count)
    lift = conf / s
    if conf == 1.0:
        conv = float('inf')
    else:
        conv = (1-s) / (1-conf)
    return [conf, lift, conv]

In [41]:
def generate_rules(measure, supps, transaction_count):
    rules = []
    CONF_THRESHOLD = 0.4
    LIFT_THRESHOLD = 20.0
    CONV_THRESHOLD = 5.0
    if measure == 'conf':
        for i in range(2, len(supps)+1):
            for k,v in supps[i-1].items():
                k = k.split(',')
                for j in range(1, len(k)):
                    for a in itertools.combinations(k, j):
                        b = tuple([w for w in k if w not in a])
                        [conf, lift, conv] = measures(v,
                                supps[len(a)-1][','.join(a)],
                                supps[len(b)-1][','.join(b)],
                                transaction_count)
                        if conf >= CONF_THRESHOLD:
                            rules.append((a, b, conf, lift, conv))
            rules = sorted(rules, key=lambda x: (x[0], x[1]))
            rules = sorted(rules, key=lambda x: (x[2]), reverse=True)
    elif measure == 'lift':
        for i in range(2, len(supps)+1):
            for k,v in supps[i-1].items():
                k = k.split(',')
                for j in range(1, len(k)):
                    for a in itertools.combinations(k, j):
                        b = tuple([w for w in k if w not in a])
                        [conf, lift, conv] = measures(v,
                                supps[len(a)-1][','.join(a)],
                                supps[len(b)-1][','.join(b)],
                                transaction_count)
                        if lift >= LIFT_THRESHOLD:
                            rules.append((a, b, conf, lift, conv))
            rules = sorted(rules, key=lambda x: (x[0], x[1]))
            rules = sorted(rules, key=lambda x: (x[3]), reverse=True)
    elif measure == 'conv':
        for i in range(2, len(supps)+1):
            for k,v in supps[i-1].items():
                k = k.split(',')
                for j in range(1, len(k)):
                    for a in itertools.combinations(k, j):
                        b = tuple([w for w in k if w not in a])
                        [conf, lift, conv] = measures(v,
                                supps[len(a)-1][','.join(a)],
                                supps[len(b)-1][','.join(b)],
                                transaction_count)
                        if conv >= CONV_THRESHOLD:
                            rules.append((a, b, conf, lift, conv))
            rules = sorted(rules, key=lambda x: (x[0], x[1]))
            rules = sorted(rules, key=lambda x: (x[4]), reverse=True)
    else:
        for i in range(2, len(supps)+1):
            for k,v in supps[i-1].items():
                k = k.split(',')
                for j in range(1, len(k)):
                    for a in itertools.combinations(k, j):
                        b = tuple([w for w in k if w not in a])
                        [conf, lift, conv] = measures(v,
                                supps[len(a)-1][','.join(a)],
                                supps[len(b)-1][','.join(b)],
                                transaction_count)
                        if (conf >= CONF_THRESHOLD and
                                lift >= LIFT_THRESHOLD and
                                conv >= CONV_THRESHOLD):
                            rules.append((a, b, conf, lift, conv))
            rules = sorted(rules, key=lambda x: (x[0], x[1]))
            rules = sorted(rules, key=lambda x: (x[2],x[3],x[4]), reverse=True)
    return rules

In [46]:
def main(file, measure): 
    sentences = tokenize(file)
    print(len(sentences))
 
    start_time = time.time()
    supps = frequent_itemsets(sentences)
    end_time = time.time()
    print("Time spent finding frequent itemsets = {:.2f} seconds.".format(
          end_time - start_time))
 
    start_time = time.time()
    rules = generate_rules(measure, supps, len(sentences))
    for rule in rules:
        print(("{{{}}} -> {{{}}}, "
               "conf = {:.2f}, lift = {:.2f}, conv = {:.2f}").format(
              ', '.join(rule[0]), ', '.join(rule[1]), rule[2], rule[3], rule[4]))
    end_time = time.time()
    print("Time spent finding association rules = {:.2f} second.".format(
          end_time - start_time))

In [52]:
#measure could be all, conf, lift or conv
measure = 'conf'
main("groceries_mba.csv", measure)

9836
|C1| = 170
|L1| = 88
|C2| = 3781
|L2| = 207
|C3| = 548
|L3| = 31
Time spent finding frequent itemsets = 0.29 seconds.
{citrus_fruit, root_vegetables} -> {other_vegetables}, conf = 0.59, lift = 3.03, conv = 1.95
{root_vegetables, tropical_fruit} -> {other_vegetables}, conf = 0.58, lift = 3.02, conv = 1.94
{butter, other_vegetables} -> {whole_milk}, conf = 0.57, lift = 2.25, conv = 1.75
{root_vegetables, tropical_fruit} -> {whole_milk}, conf = 0.57, lift = 2.23, conv = 1.73
{root_vegetables, yogurt} -> {whole_milk}, conf = 0.56, lift = 2.20, conv = 1.70
{domestic_eggs, other_vegetables} -> {whole_milk}, conf = 0.55, lift = 2.16, conv = 1.66
{whipped/sour_cream, yogurt} -> {whole_milk}, conf = 0.52, lift = 2.05, conv = 1.57
{rolls/buns, root_vegetables} -> {whole_milk}, conf = 0.52, lift = 2.05, conv = 1.56
{other_vegetables, pip_fruit} -> {whole_milk}, conf = 0.52, lift = 2.03, conv = 1.54
{tropical_fruit, yogurt} -> {whole_milk}, conf = 0.52, lift = 2.02, conv = 1.54
{other_vegetab